## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.82,81,40,23.02,scattered clouds
1,1,Castro,BR,-24.7911,-50.0119,76.77,70,12,8.39,few clouds
2,2,Torbay,CA,47.6666,-52.7314,48.88,63,75,11.50,broken clouds
3,3,Lebu,CL,-37.6167,-73.6500,58.10,68,7,17.07,clear sky
4,4,Atuona,PF,-9.8000,-139.0333,79.48,74,14,24.40,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Castro,BR,-24.7911,-50.0119,76.77,70,12,8.39,few clouds
4,4,Atuona,PF,-9.8000,-139.0333,79.48,74,14,24.40,few clouds
12,12,Padang,ID,-0.9492,100.3543,75.25,83,16,4.00,few clouds
15,15,Rikitea,PF,-23.1203,-134.9692,77.41,63,15,11.72,few clouds
18,18,Mongu,ZM,-15.2484,23.1274,74.37,79,100,1.41,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
        

City_ID                142
City                   142
Country                142
Lat                    142
Lng                    142
Max Temp               142
Humidity               142
Cloudiness             142
Wind Speed             142
Current Description    142
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Castro,BR,76.77,few clouds,-24.7911,-50.0119,
4,Atuona,PF,79.48,few clouds,-9.8000,-139.0333,
12,Padang,ID,75.25,few clouds,-0.9492,100.3543,
15,Rikitea,PF,77.41,few clouds,-23.1203,-134.9692,
18,Mongu,ZM,74.37,overcast clouds,-15.2484,23.1274,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found............skipping")
        

Hotel not found............skipping
Hotel not found............skipping
Hotel not found............skipping
Hotel not found............skipping


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"] !=""]        
clean_hotel_df.count()

City                   138
Country                138
Max Temp               138
Current Description    138
Lat                    138
Lng                    138
Hotel Name             138
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))